# CSC421 Assignment 1 Search #

This notebook is based on the supporting material for topics covered in **Chapter 3 - Intelligent Agents** from the book *Artificial Intelligence: A Modern Approach.* You can consult and modify the code provided in search.py and search.ipynb for completing the assignment questions. 

## Introduction - Block World 

In this assignment we will explore search algorithm using a Block World consisting of a width by height grid of 
cells. That way you can reuse the ipythonblocks code from agents.ipynb and agents.py for visualization. A Block World navigation problem consists of finding a path from a starting location on the grid to a destination location of the grid. In each square/cell the search agent can move in 4 directions (UP, LEFT, RIGHT, DOWN) except the edges in which one of these directions is not available depending on which side of the grid the agent is on. 

In the simplest version of this problem the search agent just has to go from the starting cell to the destination cell and a solution is the sequence of actions required. Note that for this particular problem search algoithms are overkill and instead a very simple algorithm can be used to find immediately a solution. One can take the difference of the x-coordinates and then move the appropriate number of LEFT or RIGHT steps and similarly take the difference of the y-coodinates and then move the appropriate number of UP and DOWN steps. However implementing search algorithms for this problem will help you understand the concepts better and is also needed 
for the more complex version of the problem which includes obstacles. 

In the obstacle version of the problem you are also provided with the location of a number of squares (expressed as (x,y) coordinates that act as obstacles. For example if there is an obstacle at a particular location then the agent will not be able to move to that location. Now a solution is a sequence of actions that takes our agents from the starting location to the destination location that also avoids any obstacles. 

In the rest of the notebook we specify the different subproblems based on this setting that you will be working on for this assignment. 

# Question 2A (Minimum) 1 point

Write a subclass of the abstract class problem provided in search.ipynb. Similarly to how in search.ipynb the abstract problem is made concrete by GraphProblem in the case of searching a graph, you will make it concrete by writing BlockWorldProblem. 



In [ ]:
from search import *
from random import randrange as rand
import time
from notebook import psource, heatmap, gaussian_kernel, show_map, final_path_colors, display_visual, plot_NQueens
from ipythonblocks import BlockGrid

In [ ]:
# YOUR CODE GOES HERE 

class BlockWorldProblem(Problem):
    
    def __init__(self, initial=(0,0), goal=(0,0), size=(0,0), obstacles=[], grid=None):
        super().__init__(initial=initial, goal=goal)
        ''' Some where include the grid? '''
        self.size = size
        self.obstacles = set(obstacles)
        self.grid = grid
        print("Making block world problem of size {}".format(size))
    
    def actions(self, pos):
        '''Return neighbors'''
        x,y = pos
        possible_moves = [(x+1,y),(x-1,y),(x,y+1),(x,y-1)]
        legal_moves = set(possible_moves)
        ''' Get rid of all non legal moves '''
        for move in possible_moves:
            if not self.legal_move(move):
                legal_moves.remove(move)
        return list(legal_moves)

    
    def legal_move(self, coord):
        if coord in self.obstacles: return False
        if coord[0] >= self.size[0] or coord[0] < 0:return False
        if coord[1] >= self.size[1] or coord[1] < 0: return False
        return True
    
    def result(self, state, action): 
        ''' Add node to expanded '''
        return action

    def path_cost(self, cost_so_far, A, action, B):
        ''' On a grid cost of taking action is always one '''
        return cost_so_far + 1

    def h(self, node):
        return int(distance(node.state, self.goal))
    
block_world_problem = BlockWorldProblem(initial=(0,1),goal=(7,8),size=(10,10))

# Question 2B (Minimum) 1 point 

In the search.ipynb notebook there is visualization code for Breadth-First Graph search and Depth-First Graph search. Remove the graph visualization code but keep the search code the same. Change the code so that the frontier gets printed as the search algorithm iterates. For debugging purposes you can start with the Romania example first but in the final solution you should show how the frontier changes for a particular instances of the BlockWorld problem. Use the following dimensions: a 10 by 10 grid with a starting location of (0,1) and a destination location of (7,8). Notice that in our BlockWorld problem there are a lot of overlapping path reaching a node so it makes sense to use GraphSearch and keep track of what nodes have been expanded already so that they are not expanded again. 


In [ ]:
# YOUR CODE GOES HERE 
def breadth_first_search_graph(problem):
    # we use these two variables at the time of visualisations
    iterations = 0
    max_frontier = 0
    
    node = Node(problem.initial)
    iterations += 1
      
    if problem.goal_test(node.state):
        iterations += 1
        return {"iterations": iterations, "frontier": max_frontier}
    
    frontier = deque([node])
        
    # modify the color of frontier nodes to blue
    iterations += 1
        
    explored = set()
    while frontier:
        if len(frontier) > max_frontier: max_frontier = len(frontier)
        print("Frontier Length: ", len(frontier))
        node = frontier.popleft()
        iterations += 1        
        
        explored.add(node.state)     
        
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                if problem.goal_test(child.state):
                    iterations += 1
                    print("Path: ", child.path())
                    print("Iterations: ", iterations)
                    return {"iterations": iterations, "frontier": max_frontier}
                frontier.append(child)

                iterations += 1
                    
        iterations += 1
    return {"iterations": 0, "frontier": 0}



In [ ]:
breadth_first_search_graph(block_world_problem)

In [ ]:
def depth_first_search_graph(problem):
    """Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    If two paths reach a state, only use the first one. [Figure 3.7]"""
    # we use these two variables at the time of visualisations
    iterations = 0
    all_node_colors = []
    max_frontier = 0
    frontier = [(Node(problem.initial))]
    explored = set()
    
    # modify the color of frontier nodes to orange
    iterations += 1
      
    while frontier:
        print("Frontier Length: ",len(frontier))
        if len(frontier) > max_frontier: max_frontier = len(frontier)
        # Popping first node of stack
        node = frontier.pop()
        iterations += 1
        
        if problem.goal_test(node.state):

            iterations += 1

            print("Path: ", node.path())
            print("Iterations: ", iterations)
            return {"iterations": iterations, "frontier": max_frontier}
        
        explored.add(node.state)
        frontier.extend(child for child in node.expand(problem)
                        if child.state not in explored and
                        child not in frontier)
        
        for n in frontier:
            iterations += 1

        iterations += 1
        
    return {"iterations": 0, "frontier": 0}

In [ ]:
depth_first_search_graph(block_world_problem)

# Question 2C (Minimum) 1 point 

Using the code you wrote for the previous question to print the frontier as the search algorithm iterates show how the frontier changes for Greedy and A* graph search. Use the straight-line heuristic. 

In [ ]:
# YOUR CODE GOES HERE
def best_first_graph_search_for_vis(problem, f, log=False):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; if f is node.depth then we have breadth-first search.
    There is a subtlety: the line "f = memoize(f, 'f')" means that the f
    values will be cached on the nodes as they are computed. So after doing
    a best first search you can examine the f values of the path returned."""
    
    # we use these two variables at the time of visualisations
    iterations = 0
    all_node_colors = []
    nodes_explored = 0
    max_frontier = 0
    
    f = memoize(f, 'f')
    node = Node(problem.initial)
    iterations += 1
    
    if problem.goal_test(node.state):
        iterations += 1
        return({"iterations": iterations, "frontier": max_frontier})
    
    frontier = PriorityQueue('min', f)
    frontier.append(node)
    
    iterations += 1
    
    explored = set()
    while frontier:
        f_size = len(frontier)
        if f_size > max_frontier: max_frontier = f_size
        if log: print("Frontier Length: ", f_size)
        
        node = frontier.pop()
        nodes_explored += 1
        
        if problem.grid is not None: 
            problem.grid[node.state] = (255,512,0)
            problem.grid.flash()

        iterations += 1
        
        if problem.goal_test(node.state):
            iterations += 1
            print("Path: ", node.path())
            print("Iterations: ", iterations)
            print("Cost: ", node.path_cost)
            print("Nodes Explored: ", nodes_explored)
            print("Biggest Frontier", max_frontier)
            return({"iterations": iterations, "frontier": max_frontier})
        
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)
                iterations += 1
            elif child in frontier:
                incumbent = frontier[child]
                if f(child) < incumbent:
                    del frontier[incumbent]
                    frontier.append(child)
                    iterations += 1
                    
        if problem.grid is not None: 
            problem.grid[node.state] = (255,0,0)
            problem.grid.flash()
            
        iterations += 1
    return {"iterations": 0, "frontier": 0}

def astar_search_graph(problem, h=None, log=False):
    """A* search is best-first graph search with f(n) = g(n)+h(n).
    You need to specify the h function when you call astar_search, or
    else in your Problem subclass."""
    h = memoize(problem.h, 'h')
    return best_first_graph_search_for_vis(problem, lambda n:  h(n), log=log)
     

def greedy_best_first_search(problem, h=None, log=False):
    """Greedy Best-first graph search is an informative searching algorithm with f(n) = h(n).
    You need to specify the h function when you call best_first_search, or
    else in your Problem subclass."""
    h = memoize(h or problem.h, 'h')
    return best_first_graph_search_for_vis(problem, lambda n: h(n), log=log)



In [ ]:
astar_search_graph(block_world_problem,log=True)

In [ ]:
greedy_best_first_search(block_world_problem,log=True)

# Question 2D (Expected) 1 point 

Modify the problem definition to take into account a list of random obstacles expressed as (x,y) coordinates. 
Write code to generate random instances of such problems. To generate a random instances of a BlockWorld problem 
with width W and height H use the following algorith: 

1. Generate two random numbers between 0 and W-1 and 0 and H-1 to use as the starting location 
2. Similarly generate two random numbers to use as the starting location 
3. Similarly generate the coordinates of obstacles 

The number of obstacles should be specified as a density parameter which is the percentage of the total 
number of cells that have obstacles. For example in a 6 by 6 grid there are 36 blocks. A density of 0.33 
would mean that there are 12 randomly placed obstacles. 

Show with print statements that your code is working as expected. 
Use ipythonblocks (you can check agents.py and agents.ipynb for examples of usage) to display the 
randomly generated instances of the problem. 

In [ ]:
def create_world(height, width, density=1/3):
    obstacles = set([(rand(height),rand(width)) for i in range(int(width*height*density))])
    initial = (rand(height), rand(width))
    goal = (rand(height), rand(width))
    if initial in obstacles: obstacles.remove(initial)
    if goal in obstacles: obstacles.remove(goal)
    return {"obstacles": obstacles, "goal": goal, "initial": initial}

def draw_grid(world):
    global grid
    grid[:] = (123, 234, 123)
    for obstacle in world["obstacles"]:
        grid[obstacle] = color["obstacle"]
    """ Set initial and goal """
    grid[world["initial"]] = color['initial']
    grid[world["goal"]] = color['goal']

In [ ]:
from random import randrange as rand
# YOUR CODE GOES HERE 
width, height = (rand(5,16),rand(5,16))
grid = BlockGrid(width, height)

color = {
        "obstacle": (225, 225, 225),
        "initial": (0,0,0),
        "goal": (51, 255, 255),
        "visited": (253, 208, 23),
        "current": (43, 27, 23),
        }

world = create_world(height, width)

print(width,"X",height)
print("Number of Obstacles: ", len(world["obstacles"]))

draw_grid(world)
grid.show()

# Question 2E (Expected) 1 point

Visualize search algorithms using ipythonblocks with the same color conventions that are used in search.ipynb for visualizing search with the Romania map. Your code should update the color after every iteration similar to how the visualization works in search.ipynb. Show how DFS and A* graph search will look like using this visualization for a 10 by 10 grid with 0.1 density. 

In [ ]:
# YOUR CODE GOES HERE 
width, height = (rand(5,16),rand(5,16))
grid = BlockGrid(width, height,fill=(1,1,1))

world = create_world(height, width)

block_world_problem = BlockWorldProblem(
                            initial=world["initial"],
                            goal=world["goal"],
                            size=(height,width), 
                            obstacles=world["obstacles"], 
                            grid=grid)

draw_grid(world)
astar_search_graph(block_world_problem)

In [ ]:
# YOUR CODE GOES HERE 
width, height = (rand(5,16),rand(5,16))
grid = BlockGrid(width, height,fill=(1,1,1))

world = create_world(height, width)

block_world_problem = BlockWorldProblem(
                            initial=world["initial"],
                            goal=world["goal"],
                            size=(height,width), 
                            obstacles=world["obstacles"], 
                            grid=grid)

draw_grid(world)
greedy_best_first_search(block_world_problem)

# Question 2F (Expected) 1 point 

Modify the search code so that it tracks the number of nodes visited (time complexity) and the maximum size in terms of number of nodes in the frontier (space complexity) during the execution of the search algorithm. When running one instance of the problem you should get two integer numbers: the space complexity (in number of nodes) and the time complexity (in number of nodes). Run 100 randomly generated instances of 10 by 10 grids with 0.1 density and report the average measured time and space complexity for the following algorithms: 
BFGS, DFGS, Greedy GS, A* Graph Search. 

In [ ]:
# YOUR CODE GOES HERE 
greedy_results = []
astar_results = []
bfs_results = []
dfs_results = []

for i in range(100):
    world = create_world(10, 10, 1/10)
    block_world_problem = BlockWorldProblem(
                                initial=world["initial"],
                                goal=world["goal"],
                                size=(10,10), 
                                obstacles=world["obstacles"], 
                                grid=None)
    
    greedy_results.append(greedy_best_first_search(block_world_problem))
    astar_results.append(greedy_best_first_search(block_world_problem))
    dfs_results.append(depth_first_search_graph(block_world_problem))
    bfs_results.append(breadth_first_search_graph(block_world_problem))
    

In [ ]:
astar = {"iterations": 0, "frontier": 0}
greedy = {"iterations": 0, "frontier": 0}
bfs = {"iterations": 0, "frontier": 0}
dfs = {"iterations": 0, "frontier": 0}


for i in range(len(dfs_results)):
    astar["iterations"] += astar_results[i]["iterations"]
    astar["frontier"] += astar_results[i]["frontier"]
    greedy["iterations"] += greedy_results[i]["iterations"]
    greedy["frontier"] += greedy_results[i]["frontier"]
    bfs["iterations"] += bfs_results[i]["iterations"]
    bfs["frontier"] += bfs_results[i]["frontier"]
    dfs["iterations"] += dfs_results[i]["iterations"]
    dfs["frontier"] += dfs_results[i]["frontier"]
    
astar["iterations"] /= 100
astar["frontier"] /= 100
greedy["iterations"] /= 100
greedy["frontier"] /= 100
bfs["iterations"] /= 100
bfs["frontier"] /= 100
dfs["iterations"] /= 100
dfs["frontier"] /= 100
print("Astar Averages",astar)
print("Greedy Averages",greedy)
print("BFS Averages",bfs)
print("DFS Averages",dfs)

# Question 2G (Advanced) 1 point 

Implement one of the maze generation algorithms described below: https://en.wikipedia.org/wiki/Maze_generation_algorithm

Use this technique to convert your generated maze to blockwise geometry 
https://weblog.jamisbuck.org/2015/10/31/mazes-blockwise-geometry.html

Visualize your generated maze using ipythonblocks. 

In [ ]:
# YOUR CODE GOES HERE 
def create_maze(height, width):
    start = (0,0)
    end = (height - 1, width - 1)
    ''' Set every vertex to a wall '''
    walls = set([(x,y) for x in range(height) for y in range(width)])

maze = create_maze(10,10)

In [ ]:
width, height = (50,50)
grid = BlockGrid(width, height,fill=(1,1,1))
maze = create_maze(height, width)

block_world_problem = BlockWorldProblem(
                            initial=world["initial"],
                            goal=world["goal"],
                            size=(height,width), 
                            obstacles=world["obstacles"], 
                            grid=grid)

draw_grid(world)
astar_search_graph(block_world_problem)

# Question 2H (Advanced) 1 point 

Implement finding a path through generated mazes using Search and 
visualize the resulting solution using ipythonblocks. 

Do a comparison in terms of measured with tracing running time and 
space complexity of BFGS, DGFS, Greeedy GS, A* GS for 100 randomly generated mazes 
of 20 by 20 grids using two heuristics (Manhattan and Straight-Line distance). 

Show two tables (one for space and one for time complexity) with the results 
of 6 configurations (BFGS, DGFS, Greedy with Manhattan, Greedy with Straight-line, A* with Manhattan, A* with Straight-line). Which heuristics works better for the maze solving problem? 



In [ ]:
# YOUR CODE GOES HERE 